Copyright 2018, 2019 Tobias Jachowski

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import matplotlib
matplotlib.use('module://ipympl.backend_nbagg')
from matplotlib import pyplot as plt
plt.ion()

import os
import pathlib
import pims
import sys
import tifffile

# Load own packages
#home_path = str(pathlib.Path.home())
#software_path = os.path.join(home_path, 'ZMBP/software_development/python')
software_path = '/srv/software-repo'
tj_scripts_path = os.path.join(software_path, 'tj_scripts')
create_videos_path = os.path.join(tj_scripts_path, 'create_videos')

sys.path.append(create_videos_path)

from create_videos import adjust_roi_contrast, calculate_background, check_framerate, create_video, files, process_images

In [ ]:
# Calculate a background image for a given sequence of images
directory = '../images/'
prefix = 'B01'
suffix = None
extension = '.tif'
defect_roi = None  # (516.5, 246.5, 30, 30)  # center_x, center_y, lenght_x, length_y
background_image = '../images/Background_B01.tif'

fullnames = files(directory, prefix, suffix, extension)
background = calculate_background(fullnames, defect_roi=defect_roi,
                                  replace_mode='v')

tifffile.imsave(background_image, data=background, dtype='uint16')
#rc.image_background = background

In [ ]:
directory = '../images'
prefix = 'B01'
suffix = None
extension = '.tif'
background_image = None

# Adjust the contrast of many images by showing interactive widgets to
#   1. Create a contrast process object, to adjust the contrast
#   2. Create a process image object, to interactively show individual
#      images before and after processing
rc = adjust_roi_contrast(background_image=background_image)
#rc.image_background = pims.open(background_image)[0]
pi = process_images(rc, directory, prefix=prefix, suffix=suffix, extension=extension)

In [ ]:
# CREATE VIDEO WITH PREFIXES 'prefixes' CONTAINED IN THE FOLDER 'directory'
directory = '../images/'
prefixes = ['B01', 'B02']
suffix = None
extension = '.tif'
background_image = None

# Set the pixels to be bined in (x, y). Default (1, 1).
bin_by = (2,2)

# Adjust contrast and brightness
# Set the desired width of the grey values
width_grey, offset_grey = None, None
# OR, set the min max grey values. Overwrites the above width_grey and offset_grey settings
min_grey, max_grey = cp.min, cp.max

# Select start and stop index of images to be used for the video
start_image_i = None
stop_image_i = None

# Set the time upon two succesive images are considered to
# belong to two separate videos
# Set split time in s, e.g. 1.0
split_time = None
# Set regular expression to determine split time
# from filename in ms. For example: '-([0-9]*)ms-'
split_regexp = None

# Set explicitly or autodetect fps
fps = None  # set the fps of the source video
fps_speedup = 1  # factor of speedup of the video
decimate = 1  # read every `decimate` image to produce the video

# Region to be cropped of the original images
center_x = None
center_y = None
length_x = None
length_y = None

# Quality of the resulting video
# 1 to 10, defaults to 5
quality = None 

# Create a scalebar
# unit/px resolution for the scalebar
resolution = 100
# width of the scalebar in units
scalebar_width = None

# Annotate the first 100 images with 'Annotation 1'
# and the second 100 images with 'Annotation 2'
# At pixel coordinates (150, 150) and font size 16
annotations = [
    [range(0,100), (150,150), 16, 'Annotation 1'],
    [range(100,200), (150,150), 16, 'Annotation 2'],
]

# Select the type of video to be produced
videosuffix = '.mp4'

for prefix in prefixes:
    # Create the video
    create_video(directory, prefix=prefix, suffix=suffix, extension=extension,
                 start_image_i=start_image_i, stop_image_i=stop_image_i,
                 split_time=split_time, split_regexp=split_regexp,
                 fps=fps, fps_speedup=fps_speedup, decimate=decimate, quality=quality,
                 background_image=background_image, bin_by=bin_by,
                 min_grey=min_grey, max_grey=max_grey, width_grey=width_grey, offset_grey=offset_grey,
                 center_x=center_x, center_y=center_y, length_x=length_x, length_y=length_y,
                 resolution=resolution, scalebar_width=scalebar_width,
                 annotations=annotations,
                 videosuffix=videosuffix)